<a href="https://colab.research.google.com/github/Sarvesh-10/AutoVerifyOTP/blob/master/changes_for_muril.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

2

In [2]:
!nvidia-smi

Sun Apr  7 06:06:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import tensorflow as tf
conf= tf.compat.v1.ConfigProto( device_count = {'GPU': 10 })
sess = tf.compat.v1.Session(config=conf)
conf.gpu_options.allow_growth = True

In [4]:
from transformers import TFAutoModel,AutoTokenizer

In [5]:
model = TFAutoModel.from_pretrained('google/muril-base-cased')
tokenizer = AutoTokenizer.from_pretrained('google/muril-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Some layers from the model checkpoint at google/muril-base-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [6]:
import pandas as pd
df = pd.read_csv('Book1.csv')
df.head()

,Column1,Column2
0,Mahmood Ghaznavi Aor ABdali ko bhol gaya ha tu...,OAG
1,Bhai 60sal pehle desh me kya tha pehle pta kro...,CAG
2,chutiya friday ko isliye releae krte kyoki wee...,CAG
3,जय मोदीराज,CAG
4,UPA walo ne bahot kuch kr diya tha desh k liye,CAG


In [7]:
df.dropna(inplace=True)

In [8]:
X = df['Column1']
Y = df['Column2']

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,stratify=Y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,stratify=Y)

In [10]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
import re
# import spacy
# nlp = spacy.load('de_core_news_md')
def preProcess(data):
  count = 0
  for i in range(0,len(data)):
    data[i] = re.sub('http://\S+|https://\S+', '', data[i])
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    data[i] = emoji_pattern.sub(r'', data[i])  # no emoji
    data[i] = ''.join([j for j in data[i] if not j.isdigit()])  #remove the numbers from text
    list1 = data[i].split()
    new_list = []
    for word in list1:
      # doc = nlp(word)
      # word = ' '.join([x.lemma_ for x in doc])
      if word[0]=='@' or len(word)==1 or word[0]=='#':
        count += 1
        continue
      if word in stopwords.words('english') or word in stopwords.words('hinglish'):
        count += 1
        continue
      if ord(word[0]) >= 65 and ord(word[0]) <= 90:
        index = 0
        new_character = chr(ord(word[0])+32)
        word = word[:index] + new_character + word[index+1:]
        new_list.append(word)
      else:
        new_list.append(word)
    str1 = " ".join(new_list)
    str2 = ""
    for a in str1:
      if a=='?' or a=='\'' or a=='\"' or a=='.' or a=='(' or a==')' or a=='{' or a=='}' or a=='[' or a==']' or a==':' or a==',' or a==';' or a=='&' or a=='!' or a=='@':
        continue
      else:
        str2 = str2+a
    data[i] = str2
    # print(str1)
  print(count)
  return data
X_train_clean = preProcess(list(X_train))
X_test_clean = preProcess(list(X_test))
print(X_test_clean)
# tweets = preProcess(tweets)
# print(tweets)

155249
66692
['terrific alternative sleeping could rid tv get cost night we portland', 'modi ji isarail sikho  kaha gya inch sina', 'may shud break tour pak wit iyer', 'arey magarmach ankh gila kapra daal deyta aram nikal', 'tum bich bolesb vichar art- anusar', 'ghareeb satao', 'pagal maulana', 'did tata group breach rules im referring age bracket', 'अभी राजनाथ अपनी माँ चुदा रहा है उधर जवान मर रहे हैं', 'saleJawab dena hogaIn pak sena koSaleSuar logoSurf kashmir kashmirNiti bat onko ati haiSale', 'talking companies govt making economical generic drugs available patients iMA president', 'ek chor sabke saamne bhag jaate digital chor paisa rakhte chori vikas kebhakto dikhao baat papa pichhwada lAL', 'हमारे सैनिक को इतना मत बांध हो यार हमारा हिंदुस्तान का सैनिक शेर है और शेर को कुत्ते का शिकार करने के लिए सोचने की जरूरत नहीं है कुत्ता कुत्ता ही रहेगा शेर का एक पंजा ही उसके लिए काफी है hamare sainik शहीद होते हैं बहुत दुख होता है यार शहीद सैनिक शहीद होते हैं बलि देने के लिए नहीं होते हमारे 

In [12]:

max_length = 128
batch_size=32

In [13]:
train_encodings = tokenizer(X_train_clean,truncation=True,max_length=max_length,return_tensors='tf',padding='max_length')

In [14]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform labels in y_train
y_train_encoded = label_encoder.fit_transform(Y_train)

# Transform labels in y_test
y_test_encoded = label_encoder.transform(Y_test)


In [15]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train_encoded))


In [16]:
test_encodings = tokenizer(X_test_clean, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test_encoded))

In [17]:
import tensorflow as tf

# Create TensorFlow datasets from tokenized encodings and labels
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train_encoded))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test_encoded))

# Define batch size
batch_size = 32

# Shuffle and batch the training dataset
train_dataset = train_dataset.shuffle(len(train_dataset)).batch(batch_size)

# Batch the test dataset
test_dataset = test_dataset.batch(batch_size)


In [18]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
array([[   104,  14384, 174541, ...,      0,      0,      0],
       [   104,  15637,  68718, ...,      0,      0,      0],
       [   104,    184,   2597, ...,      0,      0,      0],
       ...,
       [   104,    172,   7054, ...,      0,      0,      0],
       [   104,  33330,   2575, ...,      0,      0,      0],
       [   104,  25269,   1131, ...,      0,      0,      0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 

In [19]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [20]:
classifier = BERTForClassification(model, num_classes=3)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [21]:
# Assuming you have training_data and training_labels
history = classifier.fit(
    train_dataset,
    epochs=1,
    batch_size=4)


525/525 [==============================] - 472s 774ms/step - loss: 0.9948 - accuracy: 0.4776


In [22]:
classifier.evaluate(test_dataset)

225/225 [==============================] - 72s 279ms/step - loss: 0.9224 - accuracy: 0.5356


[0.922448992729187, 0.5355555415153503]

In [ ]:
new_text = "Kaisa hai sab badiya?!!"

# Tokenize the new text data
tokenized_input = tokenizer(new_text, truncation=True, padding='max_length', max_length=max_length, return_tensors="tf")
input_ids = tokenized_input["input_ids"]
token_type_ids = tokenized_input["token_type_ids"]
attention_mask = tokenized_input["attention_mask"]
predictions = classifier.predict({"input_ids": input_ids, "token_type_ids": token_type_ids, "attention_mask": attention_mask})
probabilities = tf.nn.softmax(predictions, axis=-1)
predicted_class = tf.argmax(probabilities, axis=-1).numpy()[0]
print(predicted_class)

In [ ]:
classifier.save("./commentToxicity")